In [4]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [40]:
en(['.'])
label = en(['Sehr geehrte Kollegen! Ich danke Frau Schroedter für den fundierten Bericht.', 'Sie hat sich eingehend mit der Problematik beschäftigt und während der Behandlung im Ausschuß viele zu dem vorliegenden Bericht eingegangene nderungsanträge berücksichtigt.'])
pred = en(['Herr Kommissar, liebe Kolleginnen und Kollegen!', 'Sie hat in der Aussprache im Ausschuss viele nderungsanträge in Bezug auf diesen Bericht berücksichtigt.'])

In [44]:
label

(tensor([[    3, 18976,   873,  7392,    17,    15, 23420,    55,  1674, 12957,
             15,  7672,   180, 10363,    15,    26,   449,   218,   177,  3069,
           6305, 19146,     5,     1,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  292,     3,   547,   289,   236,  8615,    26,   181,    74,  5289,
              9,  4414, 23029,    64,     3,  5729,    74, 12196,   256,  1392,
            860,  6712,  2584,   170,   340,   426, 22646, 19146,   236, 16416,
             15,     3, 20181,     7,   288,  7921,   397, 29699,     5,     1]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [46]:
pred

(tensor([[ 8816,  5837, 11502,   291,     6, 23803, 12561,  5772,  4015,    64,
          23420,    55,     1,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0],
         [  292,     3,   547,    16,    74,  1392, 19107,   256,  1392, 18820,
           2584,     3, 20181,     7,   288,  7921,   397,    16, 20297,   219,
              3,  5162, 19146, 29699,     5,     1]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]]))

In [41]:
model = torch.load('./model/D_A.pt').eval()

In [47]:
#real sentences
x = label[0].cuda()
#delte 0 give huge diff
       
# x = torch.Tensor(x).long().cuda()
print(x)
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret =  model.classifier(distr)#(bs,1)
print(ret)

print(model(x,x_attn))

tensor([[    3, 18976,   873,  7392,    17,    15, 23420,    55,  1674, 12957,
            15,  7672,   180, 10363,    15,    26,   449,   218,   177,  3069,
          6305, 19146,     5,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  292,     3,   547,   289,   236,  8615,    26,   181,    74,  5289,
             9,  4414, 23029,    64,     3,  5729,    74, 12196,   256,  1392,
           860,  6712,  2584,   170,   340,   426, 22646, 19146,   236, 16416,
            15,     3, 20181,     7,   288,  7921,   397, 29699,     5,     1]],
       device='cuda:0')
torch.Size([2, 40])
		---------embedding
torch.Size([2, 40, 512])
tensor([[[ 11.2423,   8.0548,  14.1798,  ...,   9.8048,  -7.8672,  -3.6016],
         [-20.7424, -14.8048,  -2.7735,  ..., -33.7422,  28.2424,  27.2424],
         [ 17.9924, -13.6173,   4.1172,  ...,   4.8047,  33.2422,   1.1485],
         ...,
         [ -2.0079,   0.2159, 

In [48]:
#real sentences
x = pred[0].cuda()
       
# x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
print(torch.sum(distr,1).shape)
print(torch.sum(x_attn,1).shape)
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret =  model.classifier(distr)#(bs,1)
print(ret)

torch.Size([2, 26])
		---------embedding
torch.Size([2, 26, 512])
tensor([[[-21.7424, -25.6174,  -2.0547,  ...,  14.5548,  42.2422,  13.1173],
         [ 14.1173,  -0.1182,  11.7423,  ...,  31.9924,  13.9298,   8.4923],
         [  5.1172, -21.1174,   6.1172,  ...,  38.2422,  10.8673,  38.7422],
         ...,
         [ -2.0079,   0.2159,  -7.0860,  ...,  -0.3458,   2.6329,  -2.8829],
         [ -2.0079,   0.2159,  -7.0860,  ...,  -0.3458,   2.6329,  -2.8829],
         [ -2.0079,   0.2159,  -7.0860,  ...,  -0.3458,   2.6329,  -2.8829]],

        [[ -9.2423,  -7.9297,  17.2424,  ...,  18.8674,  17.2424,  18.3674],
         [ 11.2423,   8.0548,  14.1798,  ...,   9.8048,  -7.8672,  -3.6016],
         [ 15.1798,  -6.5234, -12.6798,  ...,   3.5547,  -7.9297,  -6.7734],
         ...,
         [-10.1798,  -5.5547,  -3.6641,  ..., -15.2423,  -1.7032,   7.8984],
         [ -4.2734,   7.3984, -14.5548,  ...,  13.0548,  -2.7266,  -3.1329],
         [ 12.6173,   8.1798, -11.6173,  ...,   7.9297,  

In [35]:
a = torch.ones(2,2)
b = torch.ones(2,1)
b[0][0]=2
print(a)
print(b)
a/b

tensor([[1., 1.],
        [1., 1.]])
tensor([[2.],
        [1.]])


tensor([[0.5000, 0.5000],
        [1.0000, 1.0000]])

In [45]:

# x_emb = model.embedding(x_)#
# print('x_emb.shape',x_emb.shape)
# distr = model.encoder(inputs_embeds=x_emb).last_hidden_state#(bs,sentence length,512)
# print('distr.shape',distr.shape)
# x_attn= x_attn.unsqueeze(-1)
# print('x_attn.shape',x_attn.shape)
# distr = torch.mul(distr,x_attn)#previously

# print('mutil',torch.sum(distr,1))
# print('count',torch.sum(x_attn,1))
# distr = torch.sum(distr,1)/torch.sum(x_attn,1)#(bs,512)
# print(torch.mean(distr,-1))
# print('mea',distr.shape)

# ret =  model.classifier(distr)#(bs,1)
# print(ret)
# ret = model.relu(ret)#(bs,1)
# print(ret)



a_pred_dis = model(x_,x_attn)
a_pred_dis

tensor([[0.9008]], device='cuda:0', grad_fn=<ReluBackward0>)

,attention_mask=x_attn is essential for the encoder inpuit


## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier